In [1]:
#import
import pandas as pd
import numpy as np
import os
from datetime import datetime

# from __future__ import division
# from __future__ import print_function
# from __future__ import absolute_import

import warnings
import itertools

# import matplotlib
# import matplotlib.pyplot as plt

# import tensorflow as tf
# from sklearn.preprocessing import MinMaxScaler, minmax_scale, QuantileTransformer, RobustScaler, StandardScaler

# import sklearn.linear_model

# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import IsolationForest

In [2]:
#define
SESSION_PATH = './sessions'
WELLNESS_PATH = './wellness'
PICKLE_PATH = './pickle/'
EXTRA_PATH = './extra/'
format = '%Y-%m-%d'

# #colonne da shiftare verso l'alto

# lista_colonne_workload = [
#     'workload-1',
#     'workload-2',
# #     'workload-3',
# #     'workload-4',
# #     'workload-5'
#                         ]
# lista_colonne_rpe = [
#     'rpe-1',
#     'rpe-2',
# #     'rpe-3',
# #     'rpe-4',
# #     'rpe-5'
# ]
# lista_colonne_Go_score = [
#     'Go_score-1',
#     'Go_score-2',
# #     'Go_score-3',
# #     'Go_score-4',
# #     'Go_score-5'
# ]

# ######################################
# lista_category = [
#     [lista_colonne_workload,'workload'],
#     [lista_colonne_rpe,'rpe'],
#     [lista_colonne_Go_score,'Go_score']
# ]
# ######################################
#colonne per DNN


#pesi per calcolare GO_Score

SLEEP_PAR = [0.1, 0.1, 0.5, 0.8, 1]
STRESS_PAR = [0.1, 0.1, 1, 1, 1]
FATIGUE_PAR = [0.1, 0.1, 0.5, 0.8, 1]
SORENESS_PAR = [0.1, 0.1, 0.5, 0.8, 1]
MOOD_PAR = [0.1, 0.1, 1, 1, 1]

In [3]:
def from_str_to_date(stringa):
    timestr = stringa.split(' ')[0]
    time = datetime.strptime(timestr, format)
    return time

def calcola_go(df):
    try:
        df['Go_score'] = SLEEP_PAR[int(df['wellness_sleep']) - 1] + STRESS_PAR[int(df['wellness_stress']) - 1] + FATIGUE_PAR[int(df['wellness_fatigue']) - 1] + SORENESS_PAR[int(df['wellness_soreness']) - 1] + MOOD_PAR[int(df['wellness_mood']) - 1]   
    except:
        pass
    return df

In [6]:
# importo csv associati a ogni player
list_files1 = os.listdir(PICKLE_PATH)
list_file = list_files1.copy()

# tolgo dalla lista dei files quelli non di player
for file in list_files1:
    if 'Player' not in file:
        list_file.remove(file)
    elif 'csv' in file:
        list_file.remove(file)

df_total = pd.DataFrame()
#per ogni file, creo e preparo il db da dare al DNN
for file in list_file:
    print(file)
    player = file.split('.')[0]
    globals()[player] = pd.read_pickle(PICKLE_PATH + file)
    
    #calcolo go score
    df_player = globals()[player].apply(calcola_go, axis=1)

    #drop delle colonne inutili
    df_player = df_player.drop(['wellness_fatigue','wellness_mood',
                                'wellness_sleep','wellness_soreness','wellness_stress','Work Ratio','rpeTl','duration'], axis=1)
    
    ######################################################################################################
#     #faccio le colonne shiftate
#     for category in lista_category:
#         i = 1
#         for column in category[0]:
#             df_player[column] = df_player[category[1]].shift(i)
#             i += 1

    
#     df_player = df_player.dropna(how='any')
    
    globals()[player] = df_player.copy()
    df_total = df_total.append(df_player, ignore_index=True)

#aggiungo la colonna mese e giorno x Evans
def calcola_mese(df):
    df['giorno'] = df['date'].day
    df['mese'] = df['date'].month
    return df

df_total = df_total.apply(calcola_mese,axis=1)

#rinomino la colonna Player Load
df_total.rename(columns={'Player Load': 'Player_Load'}, inplace=True)

#creo la colonna previsioni
lista_previsioni = ['Go_score']
for elem in lista_previsioni:
    df_total[elem + '+1'] = df_total[elem].shift(-1)


Player1.pickle
Player10.pickle
Player11.pickle
Player12.pickle
Player13.pickle
Player14.pickle
Player15.pickle
Player16.pickle
Player17.pickle
Player18.pickle
Player19.pickle
Player2.pickle
Player20.pickle
Player21.pickle
Player23.pickle
Player24.pickle
Player25.pickle
Player26.pickle
Player27.pickle
Player28.pickle
Player29.pickle
Player3.pickle
Player30.pickle
Player31.pickle
Player32.pickle
Player4.pickle
Player5.pickle
Player6.pickle
Player7.pickle
Player9.pickle


In [7]:
df_total.columns.values

array(['Go_score', 'Player_Load', 'cardioWorkload', 'date', 'exercises',
       'intensity', 'kinematicWorkload', 'mechanicalWorkload',
       'metabolicWorkload', 'perceivedWorkload', 'playerName', 'rpe',
       'workload', 'giorno', 'Go_score+1'], dtype=object)

In [8]:
df_total.to_csv(EXTRA_PATH + 'df_total.csv', sep=';')

In [9]:
df_total

,Go_score,Player_Load,cardioWorkload,date,exercises,intensity,kinematicWorkload,mechanicalWorkload,metabolicWorkload,perceivedWorkload,playerName,rpe,workload,giorno,Go_score+1
0,NaN,0.0000,0.0,2017-06-12,0,0.000000,0.000000,0.000000,0.000000,0.000000,Player1,0.0,0.000000,12,NaN
1,NaN,0.0000,0.0,2017-06-13,0,0.000000,0.000000,0.000000,0.000000,0.000000,Player1,0.0,0.000000,13,NaN
2,NaN,0.0000,0.0,2017-06-14,0,0.000000,0.000000,0.000000,0.000000,0.000000,Player1,0.0,0.000000,14,NaN
3,NaN,0.0000,0.0,2017-06-15,0,0.000000,0.000000,0.000000,0.000000,0.000000,Player1,0.0,0.000000,15,NaN
4,NaN,0.0000,0.0,2017-06-16,0,0.000000,0.000000,0.000000,0.000000,0.000000,Player1,0.0,0.000000,16,NaN
5,NaN,193.5923,0.0,2017-06-17,"[Session, Warm-up, 11vs1]",29.797157,19.203077,31.121155,15.570078,11.682068,Player1,1.0,1.035671,17,NaN
6,NaN,133.8555,0.0,2017-06-18,"[Session, Intermittent-run]",38.570188,0.000000,45.167352,11.617813,9.678427,Player1,1.0,1.374996,18,NaN
7,NaN,635.2108,0.0,2017-06-19,"[Second-half-game, First-aerobic work, First-h...",99.647969,128.824042,231.139528,110.683430,143.667122,Player1,11.0,8.036559,19,NaN
8,NaN,419.9335,0.0,2017-06-20,"[Game-11vs11, Session, First-half, Second-half...",48.788046,47.006763,110.679258,40.272964,104.434869,Player1,7.0,4.107013,20,NaN
9,NaN,85.2790,0.0,2017-06-21,[Session],18.654375,0.000000,1.481508,0.000000,4.899257,Player1,0.5,0.180961,21,NaN
